# Exploratory Data Analysis

In this notebook we will do most of the exploratory data analysis (we will do some later after we have created certain features)

## TOC

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

## Objective

Predict total sales for every product and store in the next month.

Success is measured by low error in terms of root mean squared difference

$$\displaystyle \operatorname {RMSD} ={\sqrt {\frac {\sum _{t=1}^{T}({\hat {y}}_{t}-y_{t})^{2}}{T}}}$$

**NOTE**: The `RMSE` must be clipped to `[0,20]`

## File descriptions

* `sales_train.csv` - the training set. Daily historical data from January 2013 to October 2015.
* `test.csv` - the test set. You need to forecast the sales for these shops and products for November 2015.
* `sample_submission.csv` - a sample submission file in the correct format.
* `items.csv - supplemental` information about the items/products.
* `item_categories.csv` - supplemental information about the items categories.
* `shops.csv` - supplemental information about the shops.

In [ ]:
data_dir = Path('.').absolute().joinpath('data')

sales_train = pd.read_csv(data_dir.joinpath('sales_train.csv.gz'))
sales_test = pd.read_csv(data_dir.joinpath('test.csv.gz'))
items = pd.read_csv(data_dir.joinpath('items.csv'))
item_categories = pd.read_csv(data_dir.joinpath('item_categories.csv'))
shops = pd.read_csv(data_dir.joinpath('shops.csv'))

## Loading the dataset

## Data fields

* `ID` - an Id that represents a (Shop, Item) tuple within the test set
* `shop_id` - unique identifier of a shop
* `item_id` - unique identifier of a product
* `item_category_id` - unique identifier of item category
* `item_cnt_day` - number of products sold. You are predicting a monthly amount of this measure
* `item_price` - current price of an item
* `date` - date in format dd/mm/yyyy
* `date_block_num` - a consecutive month number, used for convenience. January 2013 is 0, February 2013 is 1,..., October 2015 is 33
* `item_name` - name of item
* `shop_name` - name of shop
* `item_category_name` - name of item category

In [ ]:
print(f'sales_train has {sales_train.shape[0]} rows')
sales_train.head()

In [ ]:
print(f'sales_test has {sales_test.shape[0]} rows')
sales_test.head()

In [ ]:
print(f'items has {items.shape[0]} rows')
items.head()

In [ ]:
print(f'item_categories has {item_categories.shape[0]} rows')
item_categories.head()

In [ ]:
print(f'shops has {shops.shape[0]} rows')
shops.head()

## Check if train and test contain the same data

In [ ]:
def check_sets(train, test, col):
    """
    Checks the overlap between the column values
    
    Parameters
    ----------
    train : DataFrame
        The train data
    test : DataFrame
        The test data
    col : str
        The column to check the overlap of
    """
    
    train_col = set(train.loc[:, col].unique())
    test_col = set(test.loc[:, col].unique())

    total = len(train_col.union(test_col))
    not_in_test = len(train_col - test_col)
    not_in_train = len(test_col - train_col)

    not_in_test_frac = 100*not_in_test/total
    not_in_train_frac = 100*not_in_train/total

    print(f'{not_in_test} {col} elements are in train, but not in test ({not_in_test_frac:.1f} %)')
    print(f'{not_in_train} {col} elements are in test, but not in train ({not_in_train_frac:.1f} %)')

In [ ]:
check_sets(sales_train, sales_test, 'shop_id')

In [ ]:
check_sets(sales_train, sales_test, 'item_id')

We combine the shop ids and the item ids to check how much overlap there is between the combined features

In [ ]:
shop_item_train = sales_train.loc[:, ['shop_id','item_id']].apply(lambda cols : f'{cols[0]}_{cols[1]}', axis=1)
shop_item_test = sales_test.loc[:, ['shop_id','item_id']].apply(lambda cols : f'{cols[0]}_{cols[1]}', axis=1)

check_sets(shop_item_train.to_frame('shop_item'), shop_item_test.to_frame('shop_item'), 'shop_item')

## Investigate the split

In [ ]:
unique_train_shop = sales_train.loc[:, 'shop_id'].unique()
print(unique_train_shop)

In [ ]:
unique_test_shop = sales_test.loc[:, 'shop_id'].unique()
print(unique_test_shop)

Hard to see any patterns here, although there could be some

In [ ]:
only_in_train = [shop for shop in unique_train_shop if shop not in unique_test_shop]
print(only_in_train)

Not anything obvious here either. (Remember that the train set contains all shops)

In [ ]:
print(shop_item_train.unique()[:100])
print(shop_item_train.unique()[-100:])

In [ ]:
print(shop_item_test.unique()[:100])
print(shop_item_test.unique()[-100:])

Not really obvious that whether or not there is a pattern here.

Maybe we find something if we plot them in a 3-D plot.

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, sharex=True, sharey=True)
sales_train.plot('shop_id', 'item_id', style='.', ax=ax1)
sales_test.plot('shop_id', 'item_id', style='.', ax=ax2)
ax1.set_ylabel('item_id')
ax2.set_ylabel('item_id')
ax1.legend().set_visible(False)
ax2.legend().set_visible(False)

It appers that a stripe in the train set has been removed. Let's check that up close

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, sharex=True, sharey=True)
sales_train.loc[sales_train.loc[:, 'item_id'] > 20000].plot('shop_id', 'item_id', style='.', ax=ax1)
sales_test.loc[sales_test.loc[:, 'item_id'] > 20000].plot('shop_id', 'item_id', style='.', ax=ax2)
ax1.set_ylabel('item_id')
ax2.set_ylabel('item_id')
ax1.legend().set_visible(False)
ax2.legend().set_visible(False)

That's peculiar. This can imply that the test data is not generated completely randomly (or maybe that the train data has been partially). We can actually see several of these stripes, alebeit with smaller bands

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, sharex=True, sharey=True)
sales_train.loc[(sales_train.loc[:, 'item_id'] < 1000) & (sales_train.loc[:, 'item_id'] > 500)].plot('shop_id', 'item_id', style='.', ax=ax1)
sales_test.loc[(sales_test.loc[:, 'item_id'] < 1000) & (sales_test.loc[:, 'item_id'] > 500)].plot('shop_id', 'item_id', style='.', ax=ax2)
ax1.set_ylabel('item_id')
ax2.set_ylabel('item_id')
ax1.legend().set_visible(False)
ax2.legend().set_visible(False)

It can appear like the shop id numbers where choosen at random, and possibly also the band and location of the item id numbers in the test set

# EDA

## Missing values

In [ ]:
sales_train.isnull().sum()

## Investigate the target

In [ ]:
fig, ax = plt.subplots()
sales_train.loc[:, 'item_cnt_day'].hist(ax=ax)

Looks like we have a couple of outliners here

In [ ]:
max_item_cnt = sales_train.loc[:, 'item_cnt_day'].max()
min_item_cnt = sales_train.loc[:, 'item_cnt_day'].min()

print(f'Max item count {max_item_cnt}')
print(f'Min item count {min_item_cnt}')

### Right outliner

In [ ]:
sales_train.loc[sales_train.loc[:, 'item_cnt_day'] == max_item_cnt]

In [ ]:
ro_item_id = sales_train.loc[sales_train.loc[:, 'item_cnt_day'] == max_item_cnt, 'item_id'].values[0]
items.loc[items.loc[:, 'item_id'] == ro_item_id]

This translates to something like "Delivery to the point of issue (Boxberry)", where it from Google looks like Boxberry is doing shipping to Russia

In [ ]:
ro_item_id_cat = items.loc[items.loc[:, 'item_id'] == ro_item_id, 'item_category_id'].values[0]
item_categories.loc[item_categories.loc[:, 'item_category_id'] == ro_item_id_cat]

Which translates to delivery of goods...this may actually be a correct number

### Left outliner

In [ ]:
sales_train.loc[sales_train.loc[:, 'item_cnt_day'] == min_item_cnt]

This translates to something like "Delivery to the point of issue (Boxberry)", where it from Google looks like Boxberry is doing shipping to Russia

In [ ]:
lo_item_id = sales_train.loc[sales_train.loc[:, 'item_cnt_day'] == min_item_cnt, 'item_id'].values[0]
items.loc[items.loc[:, 'item_id'] == lo_item_id]

This translates to stickers, and (although peculiar, it could be that somebody delivered $22$ stickers)

### Clipped distribution

In [ ]:
fig, ax = plt.subplots()
sales_train.loc[sales_train.loc[:, 'item_cnt_day'] > 100, 'item_cnt_day'].hist(ax=ax, bins=200)

In [ ]:
fig, ax = plt.subplots()
sales_train.loc[sales_train.loc[:, 'item_cnt_day'] < -3, 'item_cnt_day'].hist(ax=ax, bins=200)

As these outliers are few, it should be fairly safe to replace them with the mean.

Let's investigate the clipped distribution

In [ ]:
fig, ax = plt.subplots()
sales_train.loc[(sales_train.loc[:, 'item_cnt_day'] >= 0) & 
                (sales_train.loc[:, 'item_cnt_day'] < 100),
                 'item_cnt_day'].hist(ax =ax, bins=200, log=True)

Check the value count

In [ ]:
sales_train.loc[:, 'item_cnt_day'].value_counts()

**NOTE**: Zero values are not present

**NOTE**: We are clipping the final scores to $[0, 20]$, and we can test if this is better to do before or after the prediction (for the monthly aggregated values). Nevertheless, we should treat the outliners before doing so.

## Checking that all dates are present

In [ ]:
# Converting dd.mm.yyyy to yyyy.mm.dd
sortable_date = sales_train.loc[:, 'date'].str[6:] + '.' +\
                sales_train.loc[:, 'date'].str[3:5] + '.' +\
                sales_train.loc[:, 'date'].str[:2] 

In [ ]:
max_date = sortable_date.max()
min_date = sortable_date.min()
print(f'First date in dataset: {min_date}')
print(f'Last date in dataset: {max_date}')

In [ ]:
sales_train.loc[sales_train.loc[:, 'date'] == f'{max_date[8:]}.{max_date[5:7]}.{max_date[:4]}'].head()

Cast dates in string to timedate objects

**NOTE**: It is here important to specify the format, else we get nonsensical dates

In [ ]:
sales_train_date = pd.to_datetime(sales_train.loc[:, 'date'], format='%d.%m.%Y')

Check that this is not affecting the range

In [ ]:
max_date = sales_train_date.max()
min_date = sales_train_date.min()
print(f'First date in dataset: {min_date}')
print(f'Last date in dataset: {max_date}')

In [ ]:
sales_train.loc[sales_train_date == max_date].head()

We can now safely replace the dates with datetime

In [ ]:
sales_train.loc[:, 'date'] = sales_train_date

In [ ]:
dates = pd.to_datetime(sales_train.loc[:, 'date'].unique())
date_range = pd.date_range(start=dates.min(), end=dates.max()).unique()
n_dates = len(dates)
n_date_range = len(date_range)
missing_pct = (1 - (n_dates/n_date_range))*100
print(f'{n_date_range - n_dates} days missing ({missing_pct:.1f} %)')

## Check if there are some seasonal trends

In [ ]:
fig, ax = plt.subplots()
sales_train.groupby('date')['item_cnt_day'].sum().plot()
ax.grid(True)

High sales in January, looks like the sales are not so high in 2015 compared to the previous years

In [ ]:
fig, ax = plt.subplots()
sales_train.groupby('date_block_num')['item_cnt_day'].sum().plot(ax=ax)
ax.grid(True)

As expected, the same trends are observed in the `date_block_num`

## Check the prices

In [ ]:
fig, ax = plt.subplots()
sales_train.loc[:, 'item_price'].hist(ax=ax)

In [ ]:
max_price = sales_train.loc[:, 'item_price'].max()
min_price = sales_train.loc[:, 'item_price'].min()

print(f'Max price {max_price}')
print(f'Min price {min_price}')

Looks like missing values has been encoded as $-1.0$

In [ ]:
fig, ax = plt.subplots()
sales_train.loc[sales_train.loc[:, 'item_price'] > 35000, 'item_price'].hist(ax=ax, bins=200)

$5000$ RUB was a lot of money in the time under investigation. The last point appears to be an outliner.

In [ ]:
high_prices = sales_train.loc[sales_train.loc[:, 'item_price'] > 40000].sort_values('item_price', ascending=False)
high_prices

In [ ]:
item_max_price = high_prices.iloc[0]['item_id']
items.loc[items.loc[:, 'item_id'] == item_max_price]

This translates to "Radmin 3 - 522 individuals". In other words, the price may be actual.

In [ ]:
sales_train.loc[sales_train.loc[:, 'item_id'] == item_max_price]

This was only sold once. It could be that this was a special order. One could consider to remove the point, or redo the `item_id` and change the `item_cnt_day`. 

In [ ]:
[item for item in items.loc[:, 'item_name'] if 'admin' in item]

In [ ]:
items.loc[items.loc[:, 'item_name'] == 'Radmin 3  - 1 лиц.']

We check whether this is a commonly sold product, or if it's better to just remove the data point

In [ ]:
sales_train.loc[sales_train.loc[:, 'item_id'] == 6065]

We check if these are present in the test set

In [ ]:
sales_test.loc[sales_test.loc[:, 'item_id'] == item_max_price]

In [ ]:
sales_test.loc[sales_test.loc[:, 'item_id'] == 6065]

We see that these are present in the test set, and should be taken care of

In [ ]:
second_item_max_price = high_prices.iloc[1]['item_id']
items.loc[items.loc[:, 'item_id'] == second_item_max_price]

It's possible that this translates to something like express mail service. It may that this was the actual price.

## Item id, shop id and item category

In [ ]:
fig, ax = plt.subplots()
sales_train.groupby('item_id')['item_cnt_day'].sum().plot(ax=ax, style='.')
ax.grid(True)

We probably have one dominating prodcut, let's investigate further

In [ ]:
max_item_id = sales_train.groupby('item_id')['item_cnt_day'].sum().idxmax()
items.loc[items.loc[:, 'item_id'] == max_item_id, 'item_name']

Deciding from Google images, it looks like this translates to plastic bags, so it could be that this number is in fact huge

In [ ]:
fig, ax = plt.subplots()
sales_train.groupby('shop_id')['item_cnt_day'].sum().plot(ax=ax, style='.')
ax.grid(True)

The data seem to be sane, but we note that a few shops sells a lot more then the rest

In [ ]:
fig, ax = plt.subplots()
merged_train = pd.merge(sales_train, items, how='left', on=['item_id'])
merged_train.groupby('item_category_id')['item_cnt_day'].sum().plot(ax=ax, style='.')
ax.grid(True)

As with the shop ids, it appears that a few categories are really contributing to the most of the number of items sold.

## Check for correlations

In [ ]:
fig, ax = plt.subplots()
corr = sales_train.corr()
cax = ax.matshow(corr)
fig.colorbar(cax)
_ = ax.set_xticklabels([0]+list(corr.columns.values))
_ = ax.set_yticklabels([0]+list(corr.columns.values))

In [ ]:
corr

Not suprisingly: The item id is correlated with the price

In [ ]:
fig, ax = plt.subplots()
corr = sales_test.corr()
cax = ax.matshow(corr)
fig.colorbar(cax)
_ = ax.set_xticklabels([0]+list(corr.columns.values))
_ = ax.set_yticklabels([0]+list(corr.columns.values))

In [ ]:
corr

Intrestingly enough, we see that the `shop_id` is tightly correlated with the ID number. This is a leakage we can exploit.

# TODO

* Check if ordering is important
* Check if shop_id_item_id is correlated with target
* Continue investigation of data leakage
* EDA: NA vals, empty strings, -1, very large numbers, -999999 and less, 999, 99
* Investigating the split